## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-12-00-39-24 +0000,bbc,North Korea's Benidorm-style resort welcomes f...,https://www.bbc.com/news/articles/cwyrk588dlro
1,2025-07-12-00-19-51 +0000,bbc,Trump surveys damage in Texas as search contin...,https://www.bbc.com/news/articles/cr5vlp0pmdzo
2,2025-07-12-00-07-28 +0000,bbc,The beauty and brutality of life in the shadow...,https://www.bbc.com/news/articles/cjrll874eg5o
3,2025-07-12-00-01-53 +0000,nyt,Heat Warnings Issued for Parts of Eastern Canada,https://www.nytimes.com/2025/07/11/weather/can...
4,2025-07-11-23-36-00 +0000,bbc,Flight from London to Cancun lands in US after...,https://www.bbc.com/news/articles/c98jqve2q7mo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2328/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,8
27,canada,5
106,rate,4
22,heat,3
114,charges,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
30,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,26
15,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...,24
13,2025-07-11-20-46-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...,19
5,2025-07-11-23-05-58 +0000,bbc,Canada's Carney talked tough on Trump - now so...,https://www.bbc.com/news/articles/cvg81518yyyo,19
38,2025-07-11-11-21-26 +0000,nyt,Behind Trump’s Decision to Tax Brazil to Save ...,https://www.nytimes.com/2025/07/11/world/ameri...,17


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
30,26,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
13,19,2025-07-11-20-46-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...
15,16,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...
22,16,2025-07-11-17-37-00 +0000,wsj,Nuclear power exemplifies how revamping dated ...,https://www.wsj.com/economy/trumps-unsung-econ...
10,12,2025-07-11-22-10-00 +0000,bbc,Arizona wildfires and extreme heat prompt eva...,https://www.bbc.com/news/articles/czjk10wwmkeo
31,12,2025-07-11-14-39-16 +0000,bbc,Ex-MP faces two-year wait for betting offences...,https://www.bbc.com/news/articles/c056v0m56pyo
14,11,2025-07-11-20-40-22 +0000,nyt,Son of El Chapo Pleads Guilty to Sweeping Drug...,https://www.nytimes.com/2025/07/11/us/politics...
47,11,2025-07-11-08-14-13 +0000,bbc,'I created it myself': The boy behind the vira...,https://www.bbc.com/news/articles/czxe7ey59r0o
12,10,2025-07-11-21-11-54 +0000,bbc,US court rejects plea deal for '9/11 mastermin...,https://www.bbc.com/news/articles/c78nxqqp2leo
35,10,2025-07-11-13-02-28 +0000,bbc,National Trust blames tax rises as it cuts 550...,https://www.bbc.com/news/articles/clym8029nnvo


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
